In [6]:
import os
import json
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Agent,
    MessageDeltaChunk,
    MessageDeltaTextContent,
    RunStep,
    ThreadMessage,
    ThreadRun,
)
from azure.ai.projects.models import AgentEventHandler
from azure.ai.projects.operations import AgentsOperations
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
from IPython.display import Markdown, display


import os
from typing import Any

from user_functions import user_functions

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

In [7]:
class MyEventHandler(AgentEventHandler):

    def on_message_delta(self, delta: "MessageDeltaChunk") -> None:
        for content_part in delta.delta.content:
            if isinstance(content_part, MessageDeltaTextContent):
                text_value = content_part.text.value if content_part.text else None
                yield text_value

    def on_thread_message(self, message: "ThreadMessage") -> None:
        print(f"ThreadMessage created. ID: {message.id}, Status: {message.status}")

    def on_thread_run(self, run: "ThreadRun") -> None:
        print(f"ThreadRun status: {run.status}")

        if run.status == "failed":
            print(f"Run failed. Error: {run.last_error}")

    def on_run_step(self, step: "RunStep") -> None:
        print(f"RunStep type: {step.type}, Status: {step.status}")

    def on_error(self, data: str) -> None:
        print(f"An error occurred. Data: {data}")

    def on_done(self) -> None:
        print("Stream completed.")

    def on_unhandled_event(self, event_type: str, event_data: Any) -> None:
        print(f"Unhandled Event Type: {event_type}, Data: {event_data}")


with project_client:
    functions = FunctionTool(user_functions)
    toolset = ToolSet()
    toolset.add(functions)

    agent = project_client.agents.create_agent(
        model="gpt-4o-mini", name="my-assistant", instructions="You are a helpful assistant", toolset=toolset
    )
    print(f"Created agent, ID: {agent.id}")

    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID {thread.id}")

    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hello, send an email with the datetime and weather information in New York? Also let me know the details",
    )
    print(f"Created message, message ID {message.id}")

    with project_client.agents.create_stream(
        thread_id=thread.id, assistant_id=agent.id, event_handler=MyEventHandler()
    ) as stream:
        stream.until_done()

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Convert messages.data list into a dictionary
    messages_json = json.dumps(messages.data[0].as_dict())

    messages_dict = messages.data[0]

    # Print the value
    display(Markdown(messages_dict.content[0].text['value']))

    # Print formatted json
    print(json.dumps(json.loads(messages_json), indent=4, sort_keys=True))

Created agent, ID: asst_dYydRYm6mWHLCwj3YdIg4KvS
Created thread, thread ID thread_hDlXTvRgbFRJtkU1juiYxDSm
Created message, message ID msg_Wvo39cUNhmMZMvn5pJ0IkctL
ThreadRun status: RunStatus.QUEUED
ThreadRun status: RunStatus.QUEUED
ThreadRun status: RunStatus.IN_PROGRESS
RunStep type: RunStepType.TOOL_CALLS, Status: RunStepStatus.IN_PROGRESS
RunStep type: RunStepType.TOOL_CALLS, Status: RunStepStatus.IN_PROGRESS
RunStep type: RunStepType.TOOL_CALLS, Status: RunStepStatus.COMPLETED
ThreadRun status: RunStatus.QUEUED
ThreadRun status: RunStatus.IN_PROGRESS
RunStep type: RunStepType.TOOL_CALLS, Status: RunStepStatus.IN_PROGRESS
RunStep type: RunStepType.TOOL_CALLS, Status: RunStepStatus.IN_PROGRESS
Sending email to example@example.com...
Subject: Current DateTime and Weather in New York
Body:
Current DateTime: 2024-11-13 15:17:13
Weather in New York: Sunny, 25°C
RunStep type: RunStepType.TOOL_CALLS, Status: RunStepStatus.COMPLETED
ThreadRun status: RunStatus.QUEUED
ThreadRun status: Run

I have sent the email with the following details:

- **Current DateTime**: 2024-11-13 15:17:13
- **Weather in New York**: Sunny, 25°C

The email was successfully sent to example@example.com.

{
    "assistant_id": "asst_dYydRYm6mWHLCwj3YdIg4KvS",
    "attachments": [],
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "I have sent the email with the following details:\n\n- **Current DateTime**: 2024-11-13 15:17:13\n- **Weather in New York**: Sunny, 25\u00b0C\n\nThe email was successfully sent to example@example.com."
            },
            "type": "text"
        }
    ],
    "created_at": 1731539840,
    "id": "msg_2MNUqRElDsX7ot9Ohb0nNQwn",
    "metadata": {},
    "object": "thread.message",
    "role": "assistant",
    "run_id": "run_aXBmjTbgWO8NN725btcCFvJU",
    "thread_id": "thread_hDlXTvRgbFRJtkU1juiYxDSm"
}
